$bps=8, fps=25$
<br>
$BitIndex=(0 \to FrameCounts) \frac{25}{8}$

根据时间提取水印，每秒插入一byte，当采样率变化时，插入水印的帧不会随之变化。
首先计算出提取水印所需的总视频时长，然后根据该时长生成水印索引，最后根据该索引循环将 bit 嵌入视频帧。
提取水印时同样按照上面的方法先对视频生成水印bit索引，然后根据 bit 索引计算视频帧中的 bit 和已知 bit 的相关性。
最终求得相关性最高的一组数据，即为最终的视频水印。
这里需要已知原视频水印的大小，并需要在水印插入时提供一个固定的前缀。

In [100]:
import matplotlib.pyplot as plt
import numpy as np
from models.hamming_code import encode, decode

def get_encoded_data_index(src_data_bit_size, shuffle_seed=None):
    """get index of embedded data in hamming codes, the returned idx is unordered"""
    data = np.arange(src_data_bit_size).reshape((4, -1))
    ret = np.ones((7, data.size//4), dtype=int) * -1
    ret[2] = data[0]
    ret[4:] = data[1:]
    ret = ret.flatten()

    # shuffle bits
    if shuffle_seed is not None:
        seed = int.from_bytes(md5(shuffle_seed).digest(), 'little') % 2**32
        rand = np.random.RandomState(seed)
        rand.shuffle(ret)

    idx = np.argwhere(ret >= 0).flatten()
    idx = np.array(sorted(idx, key=lambda i: ret[i]))
    return idx

def parse_watermark(wm_restore, wm, wm_prefix_bits, bps, fps, pwd=b'123456'):
    """Confidence can be estimated by the dividing of max correlativity var and mean var """
    if isinstance(wm, bytes):
        wm_size = len(wm) * 8
    elif isinstance(wm, np.ndarray):
        wm_size = wm.size
    else:
        raise ValueError('wm only support bytes and ndarray')

    # get embedded watermark sequence
    wm = encode(wm).astype(float)
    wm[wm == 0] = -1
    idx = get_encoded_data_index(wm_size, shuffle_seed=pwd)
    idx = idx[:wm_prefix_bits]
    wm[idx] *= 10
    wm_bits_index = np.round((np.arange(wm_restore.size) * fps[1] * bps) / fps[0]).astype(int)
    wm_bits_index = wm_bits_index[wm_bits_index < wm.size]
    wm_embed = wm[wm_bits_index]

    # restore bit shifting info
    correlativity = []
    for i in range(wm_embed.size):
        tmp = np.roll(wm_embed, i)
        corr = (tmp * wm_restore[:wm_embed.size]).sum()
        correlativity.append(corr)
    i = np.argmax(correlativity)
    # get wm frame from wm stream
    wm_restore_frame = np.zeros(wm_embed.size)
    tmp = wm_restore[i:i+wm_embed.size]
    wm_restore_frame[:tmp.size] = tmp
    if tmp.size < wm_restore_frame.size:
        wm_restore_frame[-i:] = wm_restore[:i]
    # shrink wm info which was amplified by time factor during embedding
    wm_ext = np.zeros(wm.size)
    wm_ext[wm_bits_index] += wm_restore_frame
    return decode(wm_ext>0, shuffle_seed=pwd)

wm_restore = np.load('../data/wm_bits.npy')
bits_per_second = 8*1
fps = [25,1]
parse_watermark(wm_restore, b'hel00000000', bps=bits_per_second, fps=fps, wm_prefix_bits=3*8)

24


b'3\xb8\xac\xc37\n\xf6D\x8ej\x1d'

In [83]:
import numpy as np
from hashlib import md5


def get_encoded_data_index(src_data_bit_size, shuffle_seed=None):
    """get index of embedded data in hamming codes, the returned idx is unordered"""
    data = np.arange(src_data_bit_size).reshape((4, -1))
    ret = np.ones((7, data.size//4), dtype=int) * -1
    ret[2] = data[0]
    ret[4:] = data[1:]
    ret = ret.flatten()

    # shuffle bits
    if shuffle_seed is not None:
        seed = int.from_bytes(md5(shuffle_seed).digest(), 'little') % 2**32
        rand = np.random.RandomState(seed)
        rand.shuffle(ret)

    idx = np.argwhere(ret >= 0).flatten()
    idx = np.array(sorted(idx, key=lambda i: ret[i]))
    return idx


data = b'hello world'
pwd = b'jf93qfj9p4894fn'
pkt = encode(data, shuffle_seed=pwd)

idx = get_encoded_data_index(88, shuffle_seed=pwd)[:24]
np.packbits(pkt[idx]).tobytes()

b'hel'